# Segmenting and Clustering Neighborhoods in Toronto

# Scraping website

Transform the data on Wikipedia page into pandas dataframe

In [7]:
import requests 
import pandas as pd 
import numpy as np
import random 

from geopy.geocoders import Nominatim 

from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium 

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 79 kB 8.6 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=c79dec3c7bf8f75930a8c92b9804fda8cc7d3eb6497af41f64dbcc2318815b07
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Folium installed
Libraries imported.


In [8]:
path='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [9]:
df_table=pd.read_html(path)

type(df_table)

list

In [10]:
df_table[0].head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
df_table[0].replace('Not assigned',np.nan,inplace=True)
df_table[0].head()

,Postal Code,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [22]:
df_table[1]= df_table[0].groupby(["Postal Code", "Borough"])["Neighbourhood"].apply(", ".join).reset_index()
df_table[1].head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In this section of the notebook, I created a dataframe containing three columns and ignored cells with a borough that is Not assigned.

In [24]:
df_table[1].shape

(103, 3)

# Merging coordinates

In [14]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_46a4a570553e4e5b835098665c88b1bd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='n_esXQJu7y-l8x5dvIoE2aEh45SKQVcOBvURENHh9dDo',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_46a4a570553e4e5b835098665c88b1bd.get_object(Bucket='segmentingandclusteringneighborho-donotdelete-pr-wszsphgjeciu4r',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
df_toronto = pd.merge(df_table[1], df_data_1, how='left', left_on = 'Postal Code', right_on = 'Postal Code')

df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Explore and cluster the neighbourhood in Toronto 

Get the latitude and longitude values of Toronto.

In [27]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


Create a map of Toronto with neighborhoods superimposed on top.

In [28]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [30]:
for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

Define Foursquare Credentials and Version

In [31]:
CLIENT_ID = '24TL4CGZ5MYTQ5S423LL5OINPFPOZ5WTTYMH2T55X4YNIRWP' # your Foursquare ID
CLIENT_SECRET = '5MQBZ3XB00SJ1MATRO1MWX5VADJ2XETQXIIKT5HOWYGHJACI' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180605'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 24TL4CGZ5MYTQ5S423LL5OINPFPOZ5WTTYMH2T55X4YNIRWP
CLIENT_SECRET:5MQBZ3XB00SJ1MATRO1MWX5VADJ2XETQXIIKT5HOWYGHJACI


Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [33]:
df_toronto.loc[0, 'Neighbourhood']

'Malvern, Rouge'

Get the neighborhood's latitude and longitude values.

In [36]:
neighbourhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df_toronto.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


Now, let's get the top 100 venues that are in Malvern Rouge within a radius of 500 meters.

In [44]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id=24TL4CGZ5MYTQ5S423LL5OINPFPOZ5WTTYMH2T55X4YNIRWP&client_secret=5MQBZ3XB00SJ1MATRO1MWX5VADJ2XETQXIIKT5HOWYGHJACI&v=20180605&ll=43.6534817,-79.3839347&radius=500&limit=100'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=24TL4CGZ5MYTQ5S423LL5OINPFPOZ5WTTYMH2T55X4YNIRWP&client_secret=5MQBZ3XB00SJ1MATRO1MWX5VADJ2XETQXIIKT5HOWYGHJACI&v=20180605&ll=43.6534817,-79.3839347&radius=500&limit=100'

In [45]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fc80eed79ceb92b08bd6d50'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 89,
  'suggestedBounds': {'ne': {'lat': 43.6579817045, 'lng': -79.37772678059432},
   'sw': {'lat': 43.6489816955, 'lng': -79.39014261940568}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng'

In [46]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [47]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Indigo,Bookstore,43.653515,-79.380696
3,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684
4,CF Toronto Eaton Centre,Shopping Mall,43.654447,-79.380952


In [48]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

89 venues were returned by Foursquare.


Explore Neighborhoods in Toronto

Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [49]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url ='https://api.foursquare.com/v2/venues/explore?&client_id=24TL4CGZ5MYTQ5S423LL5OINPFPOZ5WTTYMH2T55X4YNIRWP&client_secret=5MQBZ3XB00SJ1MATRO1MWX5VADJ2XETQXIIKT5HOWYGHJACI&v=20180605&ll=43.6534817,-79.3839347&radius=500&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

This is too much, therefore we only explore boroughs that contain the word "Toronto"

In [52]:
# "denc" = [D]owntown Toronto, [E]ast Toronto, [N]orth Toronto, [C]entral Toronto
df_toronto_denc = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_denc.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [54]:
toronto_denc_venues = getNearbyVenues(names=df_toronto_denc['Neighbourhood'],
                                   latitudes=df_toronto_denc['Latitude'],
                                   longitudes=df_toronto_denc['Longitude']
                                  )
toronto_denc_venues.head()

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,The Beaches,43.676357,-79.293031,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,The Beaches,43.676357,-79.293031,Indigo,43.653515,-79.380696,Bookstore
3,The Beaches,43.676357,-79.293031,Chatime 日出茶太,43.655542,-79.384684,Bubble Tea Shop
4,The Beaches,43.676357,-79.293031,CF Toronto Eaton Centre,43.654447,-79.380952,Shopping Mall


In [56]:
toronto_denc_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,89,89,89,89,89,89
"Brockton, Parkdale Village, Exhibition Place",89,89,89,89,89,89
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",89,89,89,89,89,89
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",89,89,89,89,89,89
Central Bay Street,89,89,89,89,89,89
Christie,89,89,89,89,89,89
Church and Wellesley,89,89,89,89,89,89
"Commerce Court, Victoria Hotel",89,89,89,89,89,89
Davisville,89,89,89,89,89,89


Let's find out how many unique categories can be curated from all the returned venues

In [57]:
print('There are {} uniques categories.'.format(len(toronto_denc_venues['Venue Category'].unique())))

There are 62 uniques categories.


Analyze Each Neighborhood

In [59]:
# one hot encoding
toronto_denc_onehot = pd.get_dummies(toronto_denc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_denc_onehot['Neighbourhood'] = toronto_denc_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_denc_onehot.columns[-1]] + list(toronto_denc_onehot.columns[:-1])
toronto_denc_onehot = toronto_denc_onehot[fixed_columns]

toronto_denc_onehot.head()

,Neighbourhood,American Restaurant,Art Museum,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,...,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [60]:
toronto_denc_onehot.shape

(3471, 63)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [61]:
toronto_grouped = toronto_denc_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,American Restaurant,Art Museum,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,...,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,Berczy Park,0.022472,0.011236,0.011236,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,...,0.011236,0.022472,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,0.011236,0.011236
1,"Brockton, Parkdale Village, Exhibition Place",0.022472,0.011236,0.011236,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,...,0.011236,0.022472,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,0.011236,0.011236
2,"Business reply mail Processing Centre, South C...",0.022472,0.011236,0.011236,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,...,0.011236,0.022472,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,0.011236,0.011236
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.022472,0.011236,0.011236,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,...,0.011236,0.022472,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,0.011236,0.011236
4,Central Bay Street,0.022472,0.011236,0.011236,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,...,0.011236,0.022472,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,0.011236,0.011236
5,Christie,0.022472,0.011236,0.011236,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,...,0.011236,0.022472,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,0.011236,0.011236
6,Church and Wellesley,0.022472,0.011236,0.011236,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,...,0.011236,0.022472,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,0.011236,0.011236
7,"Commerce Court, Victoria Hotel",0.022472,0.011236,0.011236,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,...,0.011236,0.022472,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,0.011236,0.011236
8,Davisville,0.022472,0.011236,0.011236,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,...,0.011236,0.022472,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,0.011236,0.011236
9,Davisville North,0.022472,0.011236,0.011236,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,...,0.011236,0.022472,0.011236,0.011236,0.022472,0.022472,0.011236,0.011236,0.011236,0.011236


Let's confirm the new size

In [62]:
toronto_grouped.shape

(39, 63)

Let's print each neighborhood along with the top 5 most common venues

In [64]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1      Clothing Store  0.07
2               Hotel  0.03
3          Restaurant  0.02
4  Seafood Restaurant  0.02


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0         Coffee Shop  0.08
1      Clothing Store  0.07
2               Hotel  0.03
3          Restaurant  0.02
4  Seafood Restaurant  0.02


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0         Coffee Shop  0.08
1      Clothing Store  0.07
2               Hotel  0.03
3          Restaurant  0.02
4  Seafood Restaurant  0.02


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.08
1      Clothing Store  0.07
2               Hotel  0.03
3          Restaurant  0.02
4  Seafood Restaurant  0.02


----Central Bay Street----
            

Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [67]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Clothing Store,Hotel,American Restaurant,Cosmetics Shop,New American Restaurant,Office,Theater,Plaza,Diner
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Clothing Store,Hotel,American Restaurant,Cosmetics Shop,New American Restaurant,Office,Theater,Plaza,Diner
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Clothing Store,Hotel,American Restaurant,Cosmetics Shop,New American Restaurant,Office,Theater,Plaza,Diner
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Clothing Store,Hotel,American Restaurant,Cosmetics Shop,New American Restaurant,Office,Theater,Plaza,Diner
4,Central Bay Street,Coffee Shop,Clothing Store,Hotel,American Restaurant,Cosmetics Shop,New American Restaurant,Office,Theater,Plaza,Diner


Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [69]:
from sklearn.cluster import KMeans

In [87]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:7: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [90]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_grouped

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_grouped.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

Finally, let's visualize the resulting clusters

In [79]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [88]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_denc_merged['Latitude'], 
        toronto_denc_merged['Longitude'], 
        toronto_denc_merged['Neighbourhood'], 
        toronto_denc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'Cluster Labels'

Examine Clusters

Cluster 1

In [94]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

NameError: name 'toronto_merged' is not defined

Cluster 2

In [93]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

NameError: name 'toronto_merged' is not defined

Cluster 3

In [95]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

NameError: name 'toronto_merged' is not defined

In [96]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

NameError: name 'toronto_merged' is not defined

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]